In [ ]:
########################### 生成磁通密度曲线 ##############################
############# 以材料1为例，其余材料做类似处理
import numpy as  np
import pandas as pd
import torch
import matplotlib.pyplot as plt

# 指定engine为openpyxl
material_1_data = pd.read_excel('train_data.xlsx', engine='openpyxl', sheet_name='材料1')
material_1_magnetic_flux = material_1_data.iloc[:, 4:]

wave_shape_name = list(material_1_data.iloc[:, 3])
max_value = float(np.array(material_1_magnetic_flux).max())
min_value = float(np.array(material_1_magnetic_flux).min())
time_point = np.arange(1024)
for wave_index in range(material_1_magnetic_flux.shape[0]):
    if wave_shape_name[wave_index] == '正弦波':
        wave_graph_name = 'wave_graph/matrrial_1_train/0/' + str(wave_index) + '.png'
    elif wave_shape_name[wave_index] == '三角波':
        wave_graph_name = 'wave_graph/matrrial_1_train/1/' + str(wave_index) + '.png'
    elif wave_shape_name[wave_index] == '梯形波':
        wave_graph_name = 'wave_graph/matrrial_1_train/2/' + str(wave_index) + '.png'
    else:
        print('################# error !')

    magnetic_flux_data = list(material_1_magnetic_flux.iloc[wave_index, :])
    plt.figure(figsize=(6, 6))
    plt.scatter(time_point, magnetic_flux_data, color='k', 
                marker='o', linewidths=20)
    plt.ylim((min_value, max_value))
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(wave_graph_name)  # 保存当前批次的图像
    plt.close()
    

    
# 指定engine为openpyxl
material_test_data = pd.read_excel('test_data.xlsx')
material_1_test_magnetic_flux = material_test_data.iloc[:20, 4:]
for wave_index in range(material_1_test_magnetic_flux.shape[0]):
    wave_graph_name = 'wave_graph/matrrial_1_test/0/' + str(10+wave_index) + '.png'
    magnetic_flux_data = list(material_1_test_magnetic_flux.iloc[wave_index, :])
    plt.figure(figsize=(6, 6))
    plt.scatter(time_point, magnetic_flux_data, color='k', 
                marker='o', linewidths=20)
    plt.ylim((min_value, max_value))
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(wave_graph_name)  # 保存当前批次的图像
    plt.close()
    
############################## 使用VGG模型对磁通密度曲线进行分类 ##########
import numpy as np
import pandas as pd
import torch
from torch import nn
from PIL import Image
#import matplotlib.pyplot as plt
import os
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size=3,stride=1,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16,64,kernel_size=3,stride=1,padding=1)
        self.fc1 = nn.Linear(64*7*7,128)#两个池化，所以是7*7而不是14*14
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,3)
        self.dp = nn.Dropout(p=0.5)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64 * 7* 7)#将数据平整为一维的 
        x = F.relu(self.fc1(x))
#         x = self.fc3(x)
#         self.dp(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         x = F.log_softmax(x,dim=1) NLLLoss()才需要，交叉熵不需要
        return x

def output_embedding(embeddings):
    final_embeddings = np.array(embeddings[0].cpu())
    for i in range(1, len(embeddings)):
        batch_embedding = np.array(embeddings[i].cpu())
        final_embeddings = np.vstack((final_embeddings, batch_embedding))
    return final_embeddings

def output_labels(input_labels):
    final_labels = np.array(input_labels[0].cpu())
    for i in range(1, len(input_labels)):
        batch_labels = np.array(input_labels[i].cpu())
        final_labels = np.append(final_labels, batch_labels)
    return final_labels

def train():
        # 定义转换流程，将RGB图像转换为灰度图像并调整大小到28x28，归一化到[0, 1]范围
    transform_to_mnist = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # 转换为单通道灰度图像
        transforms.Resize((28, 28)),                  # 调整图像大小为28x28
        transforms.ToTensor(),                        # 转换为Tensor并归一化到[0, 1]范          
        transforms.Lambda(lambda x: 1 - x)            # 反转颜色，将白底变为黑底
    ])

    # 使用ImageFolder读取文件夹中的图片
    train_data = datasets.ImageFolder(root='wave_graph/matrrial_4_train/',
                                      transform=transform_to_mnist)

    # 加载数据
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

    # 查看数据集中的类别
    print(train_data.classes)

    dataiter = iter(train_loader)
    images, labels = next(dataiter)

    total_batchs = len(train_loader)

    net = CNN()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    train_accs = []
    train_loss = []
    test_accs = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = net.to(device)

    ALL_EMBEDDINGS = []
    ALL_LABELS = []

    for epoch in range(35):
        running_loss = 0.0
        for i,data in enumerate(train_loader, 0):#0是下标起始位置默认为0
            # data 的格式[[inputs, labels]]       
    #         inputs,labels = data
            inputs,labels = data[0].to(device), data[1].to(device)
            #初始为0，清除上个batch的梯度信息
            optimizer.zero_grad()
            #前向+后向+优化     
            outputs = net(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            if epoch == 34:
                ALL_EMBEDDINGS.append(outputs.detach())
                ALL_LABELS.append(labels.detach())

            # loss 的输出，每个一百个batch输出，平均的loss
            running_loss += loss.item()
            if i == total_batchs-1:
                print('[%d,%5d] loss :%.3f' %
                     (epoch+1,i+1,running_loss/total_batchs))
                running_loss = 0.0
            train_loss.append(loss.item())

            # 训练曲线的绘制 一个batch中的准确率
            correct = 0
            total = 0
            _, predicted = torch.max(outputs.data, 1)
            total = labels.size(0)# labels 的长度
            correct = (predicted == labels).sum().item() # 预测正确的数目
            train_accs.append(100*correct/total)
            print('预测准确率： ', 100*correct/total)

    print('Finished Training')
    PATH = './mnist_net.pth'
    torch.save(net.state_dict(), PATH)
    #return train_acc

    test_net = CNN()
    test_net.load_state_dict(torch.load(PATH))

    # 使用ImageFolder读取文件夹中的图片
    test_data = datasets.ImageFolder(root='wave_graph/matrrial_4_test/',
                                     transform=transform_to_mnist)

    # 加载数据
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

    with torch.no_grad():# 进行评测的时候网络不更新梯度
        for data in test_loader:
            images, labels = data
            outputs = test_net(images)
            _, predicted = torch.max(outputs.data, 1)
    print('################# 预测的标签： ')
    print(predicted)
    
    pd.DataFrame(train_accs).to_csv('train_accs_material_4.txt', header=None, index=False)
    pd.DataFrame(train_loss).to_csv('train_loss_material_4.txt', header=None, index=False)
    
    FINAL_EMB = output_embedding(ALL_EMBEDDINGS)
    pd.DataFrame(FINAL_EMB).to_csv('FINAL_EMB_material_4.txt', header=None, index=False)
    
    FINAL_LABELS = output_labels(ALL_LABELS)
    pd.DataFrame(FINAL_LABELS).to_csv('FINAL_LABELS_material_4.txt', header=None, index=False)

In [ ]:
########################### 第二题
import numpy as  np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

import torch.nn as nn
import torch.optim as optim

# 指定engine为openpyxl
material_1_data = pd.read_excel('train_data.xlsx', engine='openpyxl', sheet_name='材料1')

material_1_data = material_1_data[material_1_data.iloc[:, 3] == '正弦波']

all_tem_list = np.array(material_1_data.iloc[:, 0])
tem_25_index = np.argwhere(all_tem_list==25).flatten()[:10]
tem_50_index = np.argwhere(all_tem_list==50).flatten()[:10]
tem_70_index = np.argwhere(all_tem_list==70).flatten()[:10]
tem_90_index = np.argwhere(all_tem_list==90).flatten()[:10]

example_index = np.append(tem_25_index, tem_50_index)
example_index = np.append(example_index, tem_70_index)
example_index = np.append(example_index, tem_90_index)

###################### 全部训练（温度不log）#######################
all_tem_data = np.array(material_1_data.iloc[:, 0])
tem_max_val = np.max(all_tem_data)
all_tem_data = all_tem_data / tem_max_val

all_true_P = np.array(material_1_data.iloc[:, 2])
all_true_P_max_val = np.max(all_true_P)
all_normalized_true_P = all_true_P/all_true_P_max_val
all_true_P = np.log(all_normalized_true_P)

all_used_f = np.array(material_1_data.iloc[:, 1])
all_used_f_max_val = np.max(all_used_f)
all_normalized_used_f = all_used_f/all_used_f_max_val
all_used_f = np.log(all_normalized_used_f)

all_ALL_DENSITY = material_1_data.iloc[:, 4:]
all_used_B_M = all_ALL_DENSITY.max(axis=1)
all_used_B_M_max_val = np.max(all_used_B_M)
all_normalized_used_B_M = all_used_B_M/all_used_B_M_max_val
all_used_B_M = np.log(all_normalized_used_B_M)

# 转换为PyTorch张量
x_tensor = torch.Tensor(all_used_B_M)
z_tensor = torch.Tensor(all_used_f)
t_tensor = torch.Tensor(all_tem_data)
y_tensor = torch.Tensor(all_true_P)

x_tensor = x_tensor.unsqueeze(1)
y_tensor = y_tensor.unsqueeze(1)
t_tensor = t_tensor.unsqueeze(1)
z_tensor = z_tensor.unsqueeze(1)

# 将x和z合并为输入
inputs = torch.cat([x_tensor, z_tensor, t_tensor], dim=1)

x_train, x_test, y_train, y_test = train_test_split(inputs, y_tensor, test_size=0.1, random_state=42)

# 定义简单的线性神经网络
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        # 初始化一个全连接层，输入两个特征，输出一个值
        self.linear = nn.Linear(3, 1)
        
        # 强制初始化参数a和b的范围在[1, 3]和[2, 3]之间
        with torch.no_grad():
            self.linear.weight[:, 0].uniform_(1, 3)  # 限制a的范围
            self.linear.weight[:, 1].uniform_(2, 3)  # 限制b的范围

    def forward(self, x):
        return self.linear(x)

# 初始化模型
model = LinearModel()

# 损失函数: 均方误差
criterion = nn.MSELoss()

# 优化器: 随机梯度下降
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练模型
epochs = 1200
for epoch in range(epochs):
    # 前向传播
    outputs = model(x_train)
    
    # 计算损失
    loss = criterion(outputs, y_train)
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 打印损失值
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 评估模型在测试集上的表现
model.eval()  # 切换到评估模式，禁用dropout等
with torch.no_grad():
    test_outputs = model(x_test)  # 使用测试集进行预测
    test_loss = criterion(test_outputs, y_test)  # 计算测试集上的均方误差
print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")

pre_y = np.array(test_outputs.flatten())
tru_y = np.array(y_test.flatten())
r2_index = r2_score(tru_y, pre_y)
mae_index = mean_absolute_error(tru_y, pre_y)
print(f"测试集上的决定系数: {r2_index.item():.4f}")
print(f"测试集上的mae: {mae_index.item():.4f}")


example_inputs = inputs[example_index]
# 评估模型在测试集上的表现
model.eval()  # 切换到评估模式，禁用dropout等
with torch.no_grad():
    example_outputs = model(example_inputs)  # 使用测试集进行预测
    print('##################### 预测值：')
    print(np.exp(np.array(example_outputs.flatten()))*all_true_P_max_val) 

# 打印模型拟合的参数
with torch.no_grad():
    print("Learned parameters (BETA_PREDICTED, ALPHA_PREDICTED, K_PREDICTED):")
    BETA_PREDICTED, ALPHA_PREDICTED, T_PREDICTED = model.linear.weight[0].numpy()
    K_PREDICTED = model.linear.bias.item()
    print(f"beta = {BETA_PREDICTED}, alpha = {ALPHA_PREDICTED}, t = {T_PREDICTED}, k = {K_PREDICTED}")
    
    
    ############################### 全部训练（不用温度）####################
# 将x和z合并为输入
inputs = torch.cat([x_tensor, z_tensor], dim=1)

x_train, x_test, y_train, y_test = train_test_split(inputs, y_tensor, test_size=0.1, random_state=42)

# 定义简单的线性神经网络
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        # 初始化一个全连接层，输入两个特征，输出一个值
        self.linear = nn.Linear(2, 1)
        
        # 强制初始化参数a和b的范围在[1, 3]和[2, 3]之间
        with torch.no_grad():
            self.linear.weight[:, 0].uniform_(1, 3)  # 限制a的范围
            self.linear.weight[:, 1].uniform_(2, 3)  # 限制b的范围

    def forward(self, x):
        return self.linear(x)

# 初始化模型
model = LinearModel()

# 损失函数: 均方误差
criterion = nn.MSELoss()

# 优化器: 随机梯度下降
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练模型
epochs = 1000
for epoch in range(epochs):
    # 前向传播
    outputs = model(inputs)
    
    # 计算损失
    loss = criterion(outputs, y_tensor)
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 打印损失值
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 评估模型在测试集上的表现
model.eval()  # 切换到评估模式，禁用dropout等
with torch.no_grad():
    test_outputs = model(x_test)  # 使用测试集进行预测
    test_loss = criterion(test_outputs, y_test)  # 计算测试集上的均方误差
print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")

pre_y = np.array(test_outputs.flatten())
tru_y = np.array(y_test.flatten())
r2_index = r2_score(tru_y, pre_y)
mae_index = mean_absolute_error(tru_y, pre_y)
print(f"测试集上的决定系数: {r2_index.item():.4f}")
print(f"测试集上的mae: {mae_index.item():.4f}")


example_inputs = inputs[example_index]
# 评估模型在测试集上的表现
model.eval()  # 切换到评估模式，禁用dropout等
with torch.no_grad():
    example_outputs = model(example_inputs)  # 使用测试集进行预测
    print('##################### 预测值：')
    print(np.exp(np.array(example_outputs.flatten()))*all_true_P_max_val) 
        
# 打印模型拟合的参数
with torch.no_grad():
    print("Learned parameters (BETA_PREDICTED, ALPHA_PREDICTED, K_PREDICTED):")
    BETA_PREDICTED, ALPHA_PREDICTED = model.linear.weight[0].numpy()
    K_PREDICTED = model.linear.bias.item()
    print(f"beta = {BETA_PREDICTED}, alpha = {ALPHA_PREDICTED}, k = {K_PREDICTED}")
    
###################### 全部训练（温度log）#######################
all_tem_data = np.array(material_1_data.iloc[:, 0])
tem_max_val = np.max(all_tem_data)
all_tem_data = all_tem_data / tem_max_val
all_tem_data = np.log(all_tem_data)

all_true_P = np.array(material_1_data.iloc[:, 2])
all_true_P_max_val = np.max(all_true_P)
all_normalized_true_P = all_true_P/all_true_P_max_val
all_true_P = np.log(all_normalized_true_P)

all_used_f = np.array(material_1_data.iloc[:, 1])
all_used_f_max_val = np.max(all_used_f)
all_normalized_used_f = all_used_f/all_used_f_max_val
all_used_f = np.log(all_normalized_used_f)

all_ALL_DENSITY = material_1_data.iloc[:, 4:]
all_used_B_M = all_ALL_DENSITY.max(axis=1)
all_used_B_M_max_val = np.max(all_used_B_M)
all_normalized_used_B_M = all_used_B_M/all_used_B_M_max_val
all_used_B_M = np.log(all_normalized_used_B_M)


import torch.nn.functional as F

# 转换为PyTorch张量
x_tensor = torch.Tensor(all_used_B_M)
z_tensor = torch.Tensor(all_used_f)
t_tensor = torch.Tensor(all_tem_data)
y_tensor = torch.Tensor(all_true_P)

x_tensor = x_tensor.unsqueeze(1)
y_tensor = y_tensor.unsqueeze(1)
t_tensor = t_tensor.unsqueeze(1)
z_tensor = z_tensor.unsqueeze(1)

# 将x和z合并为输入
inputs = torch.cat([x_tensor, z_tensor, t_tensor], dim=1)

x_train, x_test, y_train, y_test = train_test_split(inputs, y_tensor, test_size=0.01, random_state=42)

# 定义简单的线性神经网络
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        # 初始化一个全连接层，输入两个特征，输出一个值
        self.linear = nn.Linear(3, 1)
        
        # 强制初始化参数a和b的范围在[1, 3]和[2, 3]之间
        with torch.no_grad():
            self.linear.weight[:, 0].uniform_(1, 3)  # 限制a的范围
            self.linear.weight[:, 1].uniform_(2, 3)  # 限制b的范围

    def forward(self, x):
        return self.linear(x)

# 初始化模型
model = LinearModel()

# 损失函数: 均方误差
criterion = nn.MSELoss()

# 优化器: 随机梯度下降
optimizer = optim.SGD(model.parameters(), lr=0.005)

# 训练模型
epochs = 1500
for epoch in range(epochs):
    # 前向传播
    outputs = model(x_train)
    
    # 计算损失
    loss = criterion(outputs, y_train)
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 打印损失值
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 评估模型在测试集上的表现
model.eval()  # 切换到评估模式，禁用dropout等
with torch.no_grad():
    test_outputs = model(x_test)  # 使用测试集进行预测
    test_loss = criterion(test_outputs, y_test)  # 计算测试集上的均方误差
print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")

pre_y = np.array(test_outputs.flatten())
tru_y = np.array(y_test.flatten())
r2_index = r2_score(tru_y, pre_y)
mae_index = mean_absolute_error(tru_y, pre_y)
print(f"测试集上的决定系数: {r2_index.item():.4f}")
print(f"测试集上的mae: {mae_index.item():.4f}")


example_inputs = inputs[example_index]
# 评估模型在测试集上的表现
model.eval()  # 切换到评估模式，禁用dropout等
with torch.no_grad():
    example_outputs = model(example_inputs)  # 使用测试集进行预测
    print('##################### 预测值：')
    print(np.exp(np.array(example_outputs.flatten()))*all_true_P_max_val) 

# 打印模型拟合的参数
with torch.no_grad():
    print("Learned parameters (BETA_PREDICTED, ALPHA_PREDICTED, K_PREDICTED):")
    BETA_PREDICTED, ALPHA_PREDICTED, T_PREDICTED = model.linear.weight[0].numpy()
    K_PREDICTED = model.linear.bias.item()
    print(f"beta = {BETA_PREDICTED}, alpha = {ALPHA_PREDICTED}, t = {T_PREDICTED}, k = {K_PREDICTED}")


########################## 按温度拟合 #############################
material_1_tem_25_data = material_1_data[material_1_data.iloc[:, 0] == 25]
material_1_tem_50_data = material_1_data[material_1_data.iloc[:, 0] == 50]
material_1_tem_70_data = material_1_data[material_1_data.iloc[:, 0] == 70]
material_1_tem_90_data = material_1_data[material_1_data.iloc[:, 0] == 90]

true_P_25 = np.array(material_1_tem_25_data.iloc[:, 2])
true_P_max_val_25 = np.max(true_P_25)
normalized_true_P_25 = true_P_25/true_P_max_val_25
true_P_25 = np.log(normalized_true_P_25)

used_f_25 = np.array(material_1_tem_25_data.iloc[:, 1])
used_f_max_val_25 = np.max(used_f_25)
normalized_used_f_25 = used_f_25/used_f_max_val_25
used_f_25 = np.log(normalized_used_f_25)

ALL_DENSITY_25 = material_1_tem_25_data.iloc[:, 4:]
used_B_M_25 = ALL_DENSITY_25.max(axis=1)
used_B_M_max_val_25 = np.max(used_B_M_25)
normalized_used_B_M_25 = used_B_M_25/used_B_M_max_val_25
used_B_M_25 = np.log(normalized_used_B_M_25)
used_B_M_25 = np.array(used_B_M_25)

# 转换为PyTorch张量
x_tensor_25 = torch.Tensor(used_B_M_25)
z_tensor_25 = torch.Tensor(used_f_25)
y_tensor_25 = torch.Tensor(true_P_25)

x_tensor_25 = x_tensor_25.unsqueeze(1)
y_tensor_25 = y_tensor_25.unsqueeze(1)
z_tensor_25 = z_tensor_25.unsqueeze(1)

# 将x和z合并为输入
inputs_25 = torch.cat([x_tensor_25, z_tensor_25], dim=1)

x_train_25, x_test_25, y_train_25, y_test_25 = train_test_split(inputs_25, y_tensor_25, test_size=0.1, random_state=42)
    
true_P_50 = np.array(material_1_tem_50_data.iloc[:, 2])
true_P_max_val_50 = np.max(true_P_50)
normalized_true_P_50 = true_P_50/true_P_max_val_50
true_P_50 = np.log(normalized_true_P_50)

used_f_50 = np.array(material_1_tem_50_data.iloc[:, 1])
used_f_max_val_50 = np.max(used_f_50)
normalized_used_f_50 = used_f_50/used_f_max_val_50
used_f_50 = np.log(normalized_used_f_50)

ALL_DENSITY_50 = material_1_tem_50_data.iloc[:, 4:]
used_B_M_50 = ALL_DENSITY_50.max(axis=1)
used_B_M_max_val_50 = np.max(used_B_M_50)
normalized_used_B_M_50 = used_B_M_50/used_B_M_max_val_50
used_B_M_50 = np.log(normalized_used_B_M_50)
used_B_M_50 = np.array(used_B_M_50)

# 转换为PyTorch张量
x_tensor_50 = torch.Tensor(used_B_M_50)
z_tensor_50 = torch.Tensor(used_f_50)
y_tensor_50 = torch.Tensor(true_P_50)

x_tensor_50 = x_tensor_50.unsqueeze(1)
y_tensor_50 = y_tensor_50.unsqueeze(1)
z_tensor_50 = z_tensor_50.unsqueeze(1)

# 将x和z合并为输入
inputs_50 = torch.cat([x_tensor_50, z_tensor_50], dim=1)

x_train_50, x_test_50, y_train_50, y_test_50 = train_test_split(inputs_50, y_tensor_50, test_size=0.1, random_state=42)

true_P_70 = np.array(material_1_tem_70_data.iloc[:, 2])
true_P_max_val_70 = np.max(true_P_70)
normalized_true_P_70 = true_P_70/true_P_max_val_70
true_P_70 = np.log(normalized_true_P_70)

used_f_70 = np.array(material_1_tem_70_data.iloc[:, 1])
used_f_max_val_70 = np.max(used_f_70)
normalized_used_f_70 = used_f_70/used_f_max_val_70
used_f_70 = np.log(normalized_used_f_70)

ALL_DENSITY_70 = material_1_tem_70_data.iloc[:, 4:]
used_B_M_70 = ALL_DENSITY_70.max(axis=1)
used_B_M_max_val_70 = np.max(used_B_M_70)
normalized_used_B_M_70 = used_B_M_70/used_B_M_max_val_70
used_B_M_70 = np.log(normalized_used_B_M_70)
used_B_M_70 = np.array(used_B_M_70)

# 转换为PyTorch张量
x_tensor_70 = torch.Tensor(used_B_M_70)
z_tensor_70 = torch.Tensor(used_f_70)
y_tensor_70 = torch.Tensor(true_P_70)

x_tensor_70 = x_tensor_70.unsqueeze(1)
y_tensor_70 = y_tensor_70.unsqueeze(1)
z_tensor_70 = z_tensor_70.unsqueeze(1)

# 将x和z合并为输入
inputs_70 = torch.cat([x_tensor_70, z_tensor_70], dim=1)

x_train_70, x_test_70, y_train_70, y_test_70 = train_test_split(inputs_70, y_tensor_70, test_size=0.1, random_state=42)

true_P_90 = np.array(material_1_tem_90_data.iloc[:, 2])
true_P_max_val_90 = np.max(true_P_90)
normalized_true_P_90 = true_P_90/true_P_max_val_90
true_P_90 = np.log(normalized_true_P_90)

used_f_90 = np.array(material_1_tem_90_data.iloc[:, 1])
used_f_max_val_90 = np.max(used_f_90)
normalized_used_f_90 = used_f_90/used_f_max_val_90
used_f_90 = np.log(normalized_used_f_90)

ALL_DENSITY_90 = material_1_tem_90_data.iloc[:, 4:]
used_B_M_90 = ALL_DENSITY_90.max(axis=1)
used_B_M_max_val_90 = np.max(used_B_M_90)
normalized_used_B_M_90 = used_B_M_90/used_B_M_max_val_90
used_B_M_90 = np.log(normalized_used_B_M_90)
used_B_M_90 = np.array(used_B_M_90)

# 转换为PyTorch张量
x_tensor_90 = torch.Tensor(used_B_M_90)
z_tensor_90 = torch.Tensor(used_f_90)
y_tensor_90 = torch.Tensor(true_P_90)

x_tensor_90 = x_tensor_90.unsqueeze(1)
y_tensor_90 = y_tensor_90.unsqueeze(1)
z_tensor_90 = z_tensor_90.unsqueeze(1)

# 将x和z合并为输入
inputs_90 = torch.cat([x_tensor_90, z_tensor_90], dim=1)

x_train_90, x_test_90, y_train_90, y_test_90 = train_test_split(inputs_90, y_tensor_90, test_size=0.1, random_state=42)

# 定义简单的线性神经网络
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        # 初始化一个全连接层，输入两个特征，输出一个值
        self.linear = nn.Linear(2, 1)
        
        # 强制初始化参数a和b的范围在[1, 3]和[2, 3]之间
        with torch.no_grad():
            self.linear.weight[:, 0].uniform_(1, 3)  # 限制a的范围
            self.linear.weight[:, 1].uniform_(2, 3)  # 限制b的范围

    def forward(self, x):
        return self.linear(x)

def train_and_test(X_TRAIN, Y_TRAIN, 
                   X_TEST, Y_TEST, 
                   INPUT_1, OUTPUT_1, 
                   INPUT_2, OUTPUT_2, 
                   INPUT_3, OUTPUT_3):
    # 初始化模型
    model = LinearModel()

    # 损失函数: 均方误差
    criterion = nn.MSELoss()

    # 优化器: 随机梯度下降
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # 训练模型
    epochs = 1000
    for epoch in range(epochs):
        # 前向传播
        outputs = model(X_TRAIN)

        # 计算损失
        loss = criterion(outputs, Y_TRAIN)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 打印损失值
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    # 评估模型在测试集上的表现
    model.eval()  # 切换到评估模式，禁用dropout等
    with torch.no_grad():
        test_outputs = model(X_TEST)  # 使用测试集进行预测
        test_loss = criterion(test_outputs, Y_TEST)  # 计算测试集上的均方误差
    print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")
    
    
    # 评估模型在测试集上的表现
    model.eval()  # 切换到评估模式，禁用dropout等
    with torch.no_grad():
        test_outputs = model(INPUT_1)  # 使用测试集进行预测
        test_loss = criterion(test_outputs, OUTPUT_1)  # 计算测试集上的均方误差
    print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")
    
    # 评估模型在测试集上的表现
    model.eval()  # 切换到评估模式，禁用dropout等
    with torch.no_grad():
        test_outputs = model(INPUT_2)  # 使用测试集进行预测
        test_loss = criterion(test_outputs, OUTPUT_2)  # 计算测试集上的均方误差
    print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")
    
    # 评估模型在测试集上的表现
    model.eval()  # 切换到评估模式，禁用dropout等
    with torch.no_grad():
        test_outputs = model(INPUT_3)  # 使用测试集进行预测
        test_loss = criterion(test_outputs, OUTPUT_3)  # 计算测试集上的均方误差
    print(f"测试集上的损失（均方误差）: {test_loss.item():.4f}")

    # 打印模型拟合的参数
    with torch.no_grad():
        print("Learned parameters (BETA_PREDICTED, ALPHA_PREDICTED, K_PREDICTED):")
        BETA_PREDICTED, ALPHA_PREDICTED = model.linear.weight[0].numpy()
        K_PREDICTED = model.linear.bias.item()
        print(f"beta = {BETA_PREDICTED}, alpha = {ALPHA_PREDICTED}, k = {K_PREDICTED}")
        
    train_and_test(x_train_25, y_train_25, 
               x_test_25, y_test_25, 
               inputs_50, y_tensor_50, 
               inputs_70, y_tensor_70, 
               inputs_90, y_tensor_90)
    
    train_and_test(x_train_50, y_train_50, 
               x_test_50, y_test_50,
               inputs_25, y_tensor_25, 
               inputs_70, y_tensor_70, 
               inputs_90, y_tensor_90)
    
    train_and_test(x_train_70, y_train_70, 
               x_test_70, y_test_70, 
               inputs_25, y_tensor_25, 
               inputs_50, y_tensor_50, 
               inputs_90, y_tensor_90)
    
    train_and_test(x_train_90, y_train_90, 
               x_test_90, y_test_90, 
               inputs_25, y_tensor_25, 
               inputs_50, y_tensor_50,
               inputs_70, y_tensor_70)

In [ ]:
######################### 第四题 #####################################
###################### 不进行特征提取
import pandas as pd
import numpy as np

material_1_data = pd.read_excel('train_data.xlsx', engine='openpyxl', sheet_name='材料1')
material_2_data = pd.read_excel('train_data.xlsx', engine='openpyxl', sheet_name='材料2')
material_3_data = pd.read_excel('train_data.xlsx', engine='openpyxl', sheet_name='材料3')、
material_4_data = pd.read_excel('train_data.xlsx', engine='openpyxl', sheet_name='材料4')

material_1_feature = material_1_data.iloc[:, :2]
material_1_DENSITY = np.array(material_1_data.iloc[:, 4:])
material_1_DENSITY_MAX = material_1_DENSITY.max(axis=1)
material_1_DENSITY_MIN = material_1_DENSITY.min(axis=1)

# material_1_DENSITY_feature = np.column_stack((material_1_DENSITY_MAX, material_1_DENSITY_MIN))

material_1_feature = np.array(material_1_feature)
wave_shape = list(np.array(material_1_data.iloc[:, 3]))
wave_ont_hot_encode = []
for i in range(len(wave_shape)):
    if wave_shape[i] == '正弦波':
        wave_ont_hot_encode.append([0, 0, 1])
    elif wave_shape[i] == '三角波':
        wave_ont_hot_encode.append([0, 1, 0])
    elif wave_shape[i] == '梯形波':
        wave_ont_hot_encode.append([1, 0, 0])
    else:
        print('############# error !')
        
wave_ont_hot_encode = np.array(wave_ont_hot_encode)
material_1_feature = np.hstack((material_1_feature, wave_ont_hot_encode))
# material_1_feature = np.hstack((material_1_feature, material_1_DENSITY_feature))
material_1_feature = np.hstack((material_1_feature, material_1_DENSITY))
material_1_type_feature = np.tile([1, 0, 0, 0], (material_1_feature.shape[0], 1))
material_1_input_feature = np.hstack((material_1_type_feature, material_1_feature))

material_1_output = np.array(material_1_data.iloc[:, 2])
material_1_input_data = np.column_stack((material_1_input_feature, material_1_output))

material_2_feature = material_2_data.iloc[:, :2]
material_2_DENSITY = np.array(material_2_data.iloc[:, 4:])
material_2_DENSITY_MAX = material_2_DENSITY.max(axis=1)
material_2_DENSITY_MIN = material_2_DENSITY.min(axis=1)

# material_2_DENSITY_feature = np.column_stack((material_2_DENSITY_MAX, material_2_DENSITY_MIN))

material_2_feature = np.array(material_2_feature)
wave_shape = list(np.array(material_2_data.iloc[:, 3]))
wave_ont_hot_encode = []
for i in range(len(wave_shape)):
    if wave_shape[i] == '正弦波':
        wave_ont_hot_encode.append([0, 0, 1])
    elif wave_shape[i] == '三角波':
        wave_ont_hot_encode.append([0, 1, 0])
    elif wave_shape[i] == '梯形波':
        wave_ont_hot_encode.append([1, 0, 0])
    else:
        print('############# error !')
        
wave_ont_hot_encode = np.array(wave_ont_hot_encode)
material_2_feature = np.hstack((material_2_feature, wave_ont_hot_encode))
# material_2_feature = np.hstack((material_2_feature, material_2_DENSITY_feature))
material_2_feature = np.hstack((material_2_feature, material_2_DENSITY))
material_2_type_feature = np.tile([0, 1, 0, 0], (material_2_feature.shape[0], 1))
material_2_input_feature = np.hstack((material_2_type_feature, material_2_feature))

material_2_output = np.array(material_2_data.iloc[:, 2])
material_2_input_data = np.column_stack((material_2_input_feature, material_2_output))

material_3_feature = material_3_data.iloc[:, :2]
material_3_DENSITY = np.array(material_3_data.iloc[:, 4:])
material_3_DENSITY_MAX = material_3_DENSITY.max(axis=1)
material_3_DENSITY_MIN = material_3_DENSITY.min(axis=1)

# material_3_DENSITY_feature = np.column_stack((material_3_DENSITY_MAX, material_3_DENSITY_MIN))

material_3_feature = np.array(material_3_feature)
wave_shape = list(np.array(material_3_data.iloc[:, 3]))
wave_ont_hot_encode = []
for i in range(len(wave_shape)):
    if wave_shape[i] == '正弦波':
        wave_ont_hot_encode.append([0, 0, 1])
    elif wave_shape[i] == '三角波':
        wave_ont_hot_encode.append([0, 1, 0])
    elif wave_shape[i] == '梯形波':
        wave_ont_hot_encode.append([1, 0, 0])
    else:
        print('############# error !')
        
wave_ont_hot_encode = np.array(wave_ont_hot_encode)
material_3_feature = np.hstack((material_3_feature, wave_ont_hot_encode))
# material_3_feature = np.hstack((material_3_feature, material_3_DENSITY_feature))
material_3_feature = np.hstack((material_3_feature, material_3_DENSITY))
material_3_type_feature = np.tile([0, 0, 1, 0], (material_3_feature.shape[0], 1))
material_3_input_feature = np.hstack((material_3_type_feature, material_3_feature))

material_3_output = np.array(material_3_data.iloc[:, 2])
material_3_input_data = np.column_stack((material_3_input_feature, material_3_output))

material_4_feature = material_4_data.iloc[:, :2]
material_4_DENSITY = np.array(material_4_data.iloc[:, 4:])
material_4_DENSITY_MAX = material_4_DENSITY.max(axis=1)
material_4_DENSITY_MIN = material_4_DENSITY.min(axis=1)

# material_4_DENSITY_feature = np.column_stack((material_4_DENSITY_MAX, material_4_DENSITY_MIN))

material_4_feature = np.array(material_4_feature)
wave_shape = list(np.array(material_4_data.iloc[:, 3]))
wave_ont_hot_encode = []
for i in range(len(wave_shape)):
    if wave_shape[i] == '正弦波':
        wave_ont_hot_encode.append([0, 0, 1])
    elif wave_shape[i] == '三角波':
        wave_ont_hot_encode.append([0, 1, 0])
    elif wave_shape[i] == '梯形波':
        wave_ont_hot_encode.append([1, 0, 0])
    else:
        print('############# error !')
        
wave_ont_hot_encode = np.array(wave_ont_hot_encode)
material_4_feature = np.hstack((material_4_feature, wave_ont_hot_encode))
# material_4_feature = np.hstack((material_4_feature, material_4_DENSITY_feature))
material_4_feature = np.hstack((material_4_feature, material_4_DENSITY))

material_4_type_feature = np.tile([0, 0, 0, 1], (material_4_feature.shape[0], 1))
material_4_input_feature = np.hstack((material_4_type_feature, material_4_feature))

material_4_output = np.array(material_4_data.iloc[:, 2])
material_4_input_data = np.column_stack((material_4_input_feature, material_4_output))

all_input_data = np.vstack((material_1_input_data, material_2_input_data, material_3_input_data, material_4_input_data))
pd.DataFrame(all_input_data).to_csv('wave_data_all.txt', header=None, index=False)

In [ ]:
test_data = pd.read_excel('test_3.xlsx', engine='openpyxl')
material_type = np.array(test_data.iloc[:, 3])
wave_type = np.array(test_data.iloc[:, 4])

wave_type_ont_hot = []
for i in range(wave_type.shape[0]):
    if wave_type[i] == '正弦波':
        wave_type_ont_hot.append([0, 0, 1])
    elif wave_type[i] == '三角波':
        wave_type_ont_hot.append([0, 1, 0])
    elif wave_type[i] == '梯形波':
        wave_type_ont_hot.append([1, 0, 0])
    else:
        print('############ error!')
        
material_type_ont_hot = []
for i in range(material_type.shape[0]):
    if material_type[i] == '材料1':
        material_type_ont_hot.append([1, 0, 0, 0])
    elif material_type[i] == '材料2':
        material_type_ont_hot.append([0, 1, 0, 0])
    elif material_type[i] == '材料3':
        material_type_ont_hot.append([0, 0, 1, 0])
    elif material_type[i] == '材料4':
        material_type_ont_hot.append([0, 0, 0, 1]) 
    else:
        print('############ error!')
        
material_feature = np.array(test_data.iloc[:, 1:3])
material_DENSITY = np.array(test_data.iloc[:, 5:])
material_feature = np.hstack((material_feature, wave_type_ont_hot))
material_feature = np.hstack((material_feature, material_DENSITY))
material_test_input_feature = np.hstack((material_type_ont_hot, material_feature))

material_feature = np.array(test_data.iloc[:, 1:3])
material_DENSITY = np.array(test_data.iloc[:, 5:])
material_feature = np.hstack((material_feature, wave_type_ont_hot))
material_feature = np.hstack((material_feature, material_DENSITY))
material_test_input_feature = np.hstack((material_type_ont_hot, material_feature))
pd.DataFrame(material_test_input_feature).to_csv('test_predict_data_all.txt', header=None, index=False)

In [ ]:
############################################### 提取特征
import numpy as np
import pandas as pd
import torch
from torch import nn
from PIL import Image
#import matplotlib.pyplot as plt
import os
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1)
        self.fc1 = nn.Linear(128*7*7,256)#两个池化，所以是7*7而不是14*14
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64,3)
        self.dp = nn.Dropout(p=0.5)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 128 * 7* 7)#将数据平整为一维的 
        x = F.relu(self.fc1(x))
#         x = self.fc3(x)
#         self.dp(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         x = F.log_softmax(x,dim=1) NLLLoss()才需要，交叉熵不需要
        return x

def output_embedding(embeddings):
    final_embeddings = np.array(embeddings[0].cpu())
    for i in range(1, len(embeddings)):
        batch_embedding = np.array(embeddings[i].cpu())
        final_embeddings = np.vstack((final_embeddings, batch_embedding))
    return final_embeddings

def output_labels(input_labels):
    final_labels = np.array(input_labels[0].cpu())
    for i in range(1, len(input_labels)):
        batch_labels = np.array(input_labels[i].cpu())
        final_labels = np.append(final_labels, batch_labels)
    return final_labels

def train():
        # 定义转换流程，将RGB图像转换为灰度图像并调整大小到28x28，归一化到[0, 1]范围
    transform_to_mnist = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # 转换为单通道灰度图像
        transforms.Resize((28, 28)),                  # 调整图像大小为28x28
        transforms.ToTensor(),                        # 转换为Tensor并归一化到[0, 1]范          
        transforms.Lambda(lambda x: 1 - x)            # 反转颜色，将白底变为黑底
    ])

    # 使用ImageFolder读取文件夹中的图片
    train_data = datasets.ImageFolder(root='wave_graph/all_material_new/',
                                      transform=transform_to_mnist)

    # 加载数据
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

    # 查看数据集中的类别
    print(train_data.classes)

    dataiter = iter(train_loader)
    images, labels = next(dataiter)

    total_batchs = len(train_loader)

    net = CNN()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    train_accs = []
    train_loss = []
    test_accs = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = net.to(device)

    ALL_EMBEDDINGS = []
    ALL_LABELS = []

    for epoch in range(11):
        running_loss = 0.0
        for i,data in enumerate(train_loader, 0):#0是下标起始位置默认为0
            # data 的格式[[inputs, labels]]       
    #         inputs,labels = data
            inputs,labels = data[0].to(device), data[1].to(device)
            #初始为0，清除上个batch的梯度信息
            optimizer.zero_grad()
            #前向+后向+优化     
            outputs = net(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            if epoch == 10:
                ALL_EMBEDDINGS.append(outputs.detach())
                ALL_LABELS.append(labels.detach())

            # loss 的输出，每个一百个batch输出，平均的loss
            running_loss += loss.item()
            if i == total_batchs-1:
                print('[%d,%5d] loss :%.3f' %
                     (epoch+1,i+1,running_loss/total_batchs))
                running_loss = 0.0
            train_loss.append(loss.item())

            # 训练曲线的绘制 一个batch中的准确率
            correct = 0
            total = 0
            _, predicted = torch.max(outputs.data, 1)
            total = labels.size(0)# labels 的长度
            correct = (predicted == labels).sum().item() # 预测正确的数目
            train_accs.append(100*correct/total)
            print('预测准确率： ', 100*correct/total)

    print('Finished Training')
    PATH = './mnist_net.pth'
    torch.save(net.state_dict(), PATH)

In [ ]:
######################################### 拼接特征（以材料1为例）
file_name = pd.read_csv('ALL_FILES_NAME.txt', header=None)
file_name = np.array(file_name)
feature_embedding = np.array(pd.read_csv('FINAL_EMBS_ALL.txt', header=None))

material_1_feature = material_1_data.iloc[:, :2]
material_1_feature = np.array(material_1_feature)
wave_shape = list(np.array(material_1_data.iloc[:, 3]))
wave_ont_hot_encode = []
for i in range(len(wave_shape)):
    if wave_shape[i] == '正弦波':
        wave_ont_hot_encode.append([0, 0, 1])
    elif wave_shape[i] == '三角波':
        wave_ont_hot_encode.append([0, 1, 0])
    elif wave_shape[i] == '梯形波':
        wave_ont_hot_encode.append([1, 0, 0])
    else:
        print('############# error !')

wave_ont_hot_encode = np.array(wave_ont_hot_encode)
material_1_feature = np.hstack((material_1_feature, wave_ont_hot_encode))
material_1_type_feature = np.tile([1, 0, 0, 0], (material_1_feature.shape[0], 1))
material_1_feature = np.hstack((material_1_type_feature, material_1_feature))

material_1_wave_feature = np.zeros((material_1_data.shape[0], 3))
for i in range(file_name.shape[0]):
    if file_name[i][0].split('_')[1] == '1':
        file_index = int(file_name[i][0].split('_')[-1].split('.')[0])
        material_1_wave_feature[file_index] = feature_embedding[i]
        
material_1_inputs = np.hstack((material_1_feature, material_1_wave_feature))

material_1_output = np.array(material_1_data.iloc[:, 2])

material_1_input_data = np.column_stack((material_1_inputs, material_1_output))

all_input_data = np.vstack((material_1_input_data, material_2_input_data, material_3_input_data, material_4_input_data))
pd.DataFrame(all_input_data).to_csv('data_wave_feature.txt', header=None, index=False)

In [ ]:
############################ 预测材料的属性
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd

def train():
    
    test_data = pd.read_excel('test_3.xlsx', engine='openpyxl')
    material_type = np.array(test_data.iloc[:, 3])
    wave_type = np.array(test_data.iloc[:, 4])

    wave_type_ont_hot = []
    for i in range(wave_type.shape[0]):
        if wave_type[i] == '正弦波':
            wave_type_ont_hot.append([0, 0, 1])
        elif wave_type[i] == '三角波':
            wave_type_ont_hot.append([0, 1, 0])
        elif wave_type[i] == '梯形波':
            wave_type_ont_hot.append([1, 0, 0])
        else:
            print('############ error!')

    material_type_ont_hot = []
    for i in range(material_type.shape[0]):
        if material_type[i] == '材料1':
            material_type_ont_hot.append([1, 0, 0, 0])
        elif material_type[i] == '材料2':
            material_type_ont_hot.append([0, 1, 0, 0])
        elif material_type[i] == '材料3':
            material_type_ont_hot.append([0, 0, 1, 0])
        elif material_type[i] == '材料4':
            material_type_ont_hot.append([0, 0, 0, 1]) 
        else:
            print('############ error!')

    material_feature = np.array(test_data.iloc[:, 1:3])
    material_DENSITY = np.array(test_data.iloc[:, 5:])
    material_feature = np.hstack((material_feature, wave_type_ont_hot))
    material_feature = np.hstack((material_feature, material_DENSITY))
    material_test_input_feature = np.hstack((material_type_ont_hot, material_feature))

    
    # Load data and generate a train/test split 
    data = np.array(pd.read_csv('wave_data_all.txt', header=None))

    x_max = np.amax(data, axis = 0)
    data[:, 4:6] = np.log(data[:, 4:6]/x_max[4:6])
    material_test_input_feature[:, 4:6] = np.log(material_test_input_feature[:, 4:6]/x_max[4:6])
    data[:, -1:] = np.log(data[:,-1:]/x_max[-1:])

    X = data[:, :-1]
    y = data[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

    # 初始化 CatBoostRegressor 模型
    model = CatBoostRegressor(
        iterations=4000,        # 迭代次数
        learning_rate=0.15,      # 学习率
        depth=6,                # 树的深度
        eval_metric='MAE',     # 使用 RMSE 作为评估指标
        random_seed=42,
        verbose=100             # 每100轮打印一次日志
    )

    # 训练模型
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

    # 预测
    y_pred = model.predict(X_test)
    #y_test_pred = model.predict(material_test_input_feature)
    # 计算均方根误差 (RMSE)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    mae = mean_absolute_error(y_test, y_pred)

    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test MAE: {mae:.4f}")
    
    #return y_test_pred